In [15]:
# movielens 100K

#SVD 알아오기
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pathlib import Path

path = Path('../../../datafile/Recsys/ml-100k')
raw_data = pd.read_csv(path /'u.data',sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})

user_id = raw_data['user'].unique().tolist()

raw_data.head()

train,test = train_test_split(raw_data,test_size=0.2,random_state=42)
R_train = pd.pivot_table(train,values='rating',index='item',columns='user')

In [45]:
train

,user,item,rating
75220,807,1411,1.0
48955,474,659,5.0
44966,463,268,4.0
13568,139,286,4.0
92727,621,751,4.0
...,...,...,...
6265,216,231,2.0
54886,343,276,5.0
76820,437,475,3.0
860,284,322,3.0


In [50]:
uilist_test = list(zip(test['user'],test['item']))
pred_list = []
for ind in uilist_test:
    pred_list.append(R_train[ind[0],ind[1]])
pred_list

KeyError: (877, 381)

In [ ]:
R_train

user,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item,,,,,,,,,,,,,,,,,,,,,
1,NaN,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_item_mat = pd.pivot_table(data=raw_data,values='rating',columns='user',index='item')
user_item_mat

user,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
np.nonzero(user_item_mat)

(array([   0,    0,    0, ..., 1681, 1681, 1681], shape=(1586126,)),
 array([  0,   1,   2, ..., 940, 941, 942], shape=(1586126,)))

In [4]:
import numpy as np

# 분해할 행렬 A 생성
A = np.array([
    [3, 1, 1],
    [-1, 3, 1]
])

print("--- 원본 행렬 A ---")
print(A)

U, s, Vt = np.linalg.svd(A)

print("\n--- 분해된 행렬들 ---")
print("U의 모양:", U.shape)
print("s의 모양:", s.shape) # 1차원 배열임에 주의!
print("Vt의 모양:", Vt.shape)

# s를 대각 행렬 Σ로 재구성
# 원본 A의 크기에 맞게 0을 채워줌
Sigma = np.zeros(A.shape)
Sigma[:len(s), :len(s)] = np.diag(s)

k = 2
U_k = U[:,:k]
Sigma_k = Sigma[:k,:k]
Vt_k = Vt[:k,:]

print("\n--- 재구성된 대각 행렬 Σ ---")
print(Sigma)

# U, Σ, Vt를 다시 곱해서 A를 복원
A_restored = U @ Sigma @ Vt  # @는 행렬 곱셈 연산자
A_k_restored = U_k @ Sigma_k @ Vt_k

print("\n--- 복원된 행렬 A,A_k ---")
print('Truncated SVD: ',A_k_restored.round(3))
print('Full-SVD : ',A_restored.round(3)) # 부동소수점 오차를 감안해 반올림

--- 원본 행렬 A ---
[[ 3  1  1]
 [-1  3  1]]

--- 분해된 행렬들 ---
U의 모양: (2, 2)
s의 모양: (2,)
Vt의 모양: (3, 3)

--- 재구성된 대각 행렬 Σ ---
[[3.46410162 0.         0.        ]
 [0.         3.16227766 0.        ]]

--- 복원된 행렬 A,A_k ---
Truncated SVD:  [[ 3.  1.  1.]
 [-1.  3.  1.]]
Full-SVD :  [[ 3.  1.  1.]
 [-1.  3.  1.]]
